# [Paris Saclay Center for Data Science](http://www.datascience-paris-saclay.fr)

## [Radar trajectories RAMP](http://www.ramp.studio/problems/radar_trajectories): classifying flying objects using radar trajectories

_Balázs Kégl (LAL/CNRS), Akin Kazakci (Mines ParisTech), Silvère Bonnabel (Mines ParisTech), Sami Jouaber (Mines ParisTech)_

## Introduction

The first goal of a traffic control center is to yield a clear vision of the state of the air traffic. This vision is based on various information about the flying aircrafts, namely radar measurements and other information provided by the planes themselves about their position, as well as some of their characteristics, through the Automatic Dependence Surveillance Broadcast (ADSB) system. To provide air traffic controllers with efficient decision making tools, one should be able to automatically detect a number of features regarding an aircraft, solely based on the radar measurements (that do not depend on whether the aircraft is cooperating or not). In particular, it is desirable to establish whether the aircraft is civilian or military, if its behavior is aggressive or not, what kind of maneuvers it is capable of performing, and which category of airplane it is (helicopter, fighter, liner, drone, etc.). Such information may prove useful for instance to adapt the tracking
algorithm of the radars to the type of aircraft (highly maneuvering aircrafts may require more radar
resource to be tracked), and more generally for decision making supporting tools.

## Type of data used and prediction task

Radar measurements are classified since they contain information about French military operations and trips. Hence, we have collected aircraft trajectories (i.e., position over time) using the publicly available ADSB data. Those trajectories serve as a publicly available alternative to the radar position measurements. The goal of the classification task is to recognize the type of flying objects. There are 19 types, labeled by 4-digit strings: '1111', '1112', '1121', '1122', '1132', '1222', '1224', '1231', '1232', '1233', '1234', '1324', '1332', '1333', '1334', '4111', '4121', '4122', '4222'. Each digit denotes an attribute
1. Species: Kind of aircraft
     - 1 = Airplane - Liner
     - 4 = Helicopter
2. WTC: Wake Turbulence Category
     - 1 = Light
     - 2 = Middle
     - 3 = Heavy
3. EngType: Kind of engine
     - 1 = Piston
     - 2 = Turboprop
     - 3 = Jet
4. Engines: Number of engines

The models will be evaluated by cross entropy a.k.a negative log likelihood (NLL)
$$
- \frac{1}{19} \sum_{i=1}^{19} \log \hat{p}_i
$$
where $\hat{p}_i$ is the predicted probability of the $i$th class given the input (radar trajectories, see below). Besides this official score, we will also display the accuracy (number of correctly classified types divided by the size of the test set) which is more human readable. The NLL score has the advantage that it incentivizes you to come up with unbiased probability estimates which can then be aggregated across aircraft types if we change the task (e.g., to predict the number of engines). 


### Requirements

* numpy>=1.10.0  
* matplotlib>=1.5.0 
* pandas>=0.19.0  
* scikit-learn>=0.18
* ramp-workflow

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

## Exploratory data analysis

Let's first import the `problem.py` script which has data reader routines.

In [2]:
import imp
module_name = str(int(1000000000 * np.random.rand()))
problem = imp.load_source('', 'problem.py')

Let's read the training data and labels from `data/train.pkl` into a pandas dataframe.

In [3]:
X_df, y_array = problem.get_train_data()
X_df

,data,trajectory_id
index,,
0,"[[142.3, 13065.2072311, -37.3482725781, 7.6629...",10
1,"[[178.1, 12293.3296794, -41.8313170148, -2.782...",10
2,"[[528.1, -2603.7653628, -173.591179302, 4.0816...",10
3,"[[204.0, 11468.8976453, -75.9557784458, -3.710...",10
4,"[[335.0, 6668.3636331, -107.36545794, 0.927818...",10
5,"[[38.9, 17597.3725596, -47.7188174498, 8.40393...",10
6,"[[264.2, 9455.57780146, -76.1110502916, -2.119...",10
7,"[[238.6, 9664.36696588, 29.6750335695, 8.19662...",10
8,"[[363.7, 5592.22368649, -125.457647118, -0.083...",10


In [4]:
y_array

array(['1111', '1111', '1111', ..., '4222', '4222', '4222'], dtype=object)

The training data has two columns. `trajectory_id` is the id of the long trajectory from which we sampled 1000 consecutive trajectory points at random starting times. To have enough training data, we took 10 (possibly overlapping) samples from each long trajectory. The trajectory itself is found in the `data` column. Each instance is represented by a 2D numpy array where each column is a trajectory feature and each row is a time point. Let us convert a single instance into a pandas table to visualize it. 

In [5]:
columns = [
    'T', 'X', 'Vx', 'Ax', 'Jx', 'Y', 'Vy','Ay', 'Jy',
    'Z', 'Vz', 'Az', 'Jz', 'U2', 'C2', 'U3', 'C3', 'T3']
i = 0
trajectory_df = pd.DataFrame(X_df['data'][i], columns=columns)
trajectory_df

,T,X,Vx,Ax,Jx,Y,Vy,Ay,Jy,Z,Vz,Az,Jz,U2,C2,U3,C3,T3
0,142.3,13065.207231,-37.348273,7.662910,0.558759,10648.565303,-81.879076,13.799308,1.480220,278.248441,0.275643,0.042042,-0.014298,89.994869,1.708234e-06,89.995291,0.000154,-0.000016
1,142.4,13061.548323,-35.959268,7.698060,0.532016,10640.499794,-79.613038,13.917953,1.442313,278.276444,0.283573,0.040485,-0.014462,87.357340,1.929820e-06,87.357800,0.000169,-0.000015
2,142.5,13058.059018,-34.561718,7.729593,0.505136,10632.705141,-77.324428,14.031347,1.404058,278.305394,0.291147,0.038914,-0.014616,84.696987,2.190796e-06,84.697487,0.000186,-0.000014
3,142.6,13054.741148,-33.156487,7.757482,0.478135,10625.185351,-75.014413,14.139425,1.365475,278.335239,0.298362,0.037330,-0.014762,82.015333,2.499874e-06,82.015876,0.000206,-0.000014
4,142.7,13051.596381,-31.744444,7.781706,0.451030,10617.944223,-72.684172,14.242129,1.326588,278.365926,0.305219,0.035735,-0.014899,79.313924,2.868085e-06,79.314511,0.000228,-0.000013
5,142.8,13048.626221,-30.326460,7.802246,0.423839,10610.985345,-70.334893,14.339404,1.287418,278.397402,0.311716,0.034130,-0.015027,76.594330,3.309504e-06,76.594965,0.000254,-0.000012
6,142.9,13045.832000,-28.903414,7.819086,0.396578,10604.312091,-67.967776,14.431199,1.247987,278.429614,0.317856,0.032515,-0.015146,73.858147,3.842247e-06,73.858831,0.000284,-0.000012
7,143.0,13043.214882,-27.476185,7.832214,0.369263,10597.927615,-65.584024,14.517467,1.208315,278.462509,0.323637,0.030893,-0.015256,71.106997,4.489832e-06,71.107733,0.000320,-0.000011
8,143.1,13040.775853,-26.045654,7.841621,0.341912,10591.834848,-63.184851,14.598164,1.168423,278.496031,0.329061,0.029265,-0.015359,68.342531,5.283088e-06,68.343323,0.000362,-0.000010
9,143.2,13038.515727,-24.612703,7.847303,0.314539,10586.036493,-60.771474,14.673252,1.128332,278.530128,0.334128,0.027631,-0.015452,65.566434,6.262841e-06,65.567285,0.000411,-0.000010


Here is the meaning of the columns:
- `T`: time
- `X`, `Vx`, `Ax`, `Jx`: X coordinate of the resp. position, velocity, acceleration, and jerk (i.e. first to third order derivative) of the aircraft
- `Y`, `Vy`, `Ay`, `Jy`: Y coordinate of the resp. position, velocity,  acceleration, and jerk (i.e. first to third order derivative) of the aircraft
- `Z`, `Vz`, `Az`, `Jz`: Z coordinate of the resp. position, velocity,  acceleration, and jerk (i.e. first to third order derivative) of the aircraft- `U2`: absolute value of velocity vector projected onto the ground plane
- `C3`: curvature of the 3D trajectory: $\frac{||P'\times P''||}{||P'||^3}$ where $P = (X, Y, Z)$
- `T3`: torsion of the 3D trajectory: $\frac{\det(P',P'',P''')}{||P'\times P''||^2}$
- `C2`: curvature of the trajectory projected onto the ground plane: $\frac{||\tilde P'\times \tilde
P''||}{||\tilde P'||^3}$ where $\tilde P=(X,Y,0)$
- `U3`: absolute value of the 3D velocity vector


## The pipeline

For submitting at the [RAMP site](http://ramp.studio), you will have to write two classes, saved in two different files:   
* the class `FeatureExtractor`, which will be used to extract features for classification from the dataset and produce a numpy array of size (number of samples $\times$ number of features). 
* a class `Classifier` to predict aircraft type

### Feature extractor

The feature extractor implements a `transform` member function. It is saved in the file [`submissions/starting_kit/feature_extractor.py`](/edit/submissions/starting_kit/feature_extractor.py). It receives the pandas dataframe `X_df` defined at the beginning of the notebook. It should produce a numpy array representing the extracted features, which will then be used for the classification. The following simple feature extractor simply computes the mean of each feature and produces an 18 dimensional feature vector for each trajectory.

In [6]:
import numpy as np

class FeatureExtractor():
    def __init__(self):
        pass

    # use this if you need to learn something at training time that depends on the labels
    # this will not be called on the test instances
    def fit(self, X_df, y):
        pass

    # this will be called both on the training and test instance
    def transform(self, X_df):
        data_matrix = np.asarray(list(X_df['data'].values))
        # taking the mean of each data matrix variable, averaging over axis 1
        means = data_matrix.mean(axis=1)
        return means


Let's try it on the training data.

In [7]:
fe = FeatureExtractor()
fe.fit(X_df, y_array)  
X_array = fe.transform(X_df)
X_array.shape

(4560, 18)

### Classifier

The classifier follows a classical scikit-learn classifier template. In its simplest form it takes a scikit-learn pipeline, assigns it to `self.clf` in `__init__`, then calls its `fit` and `predict_proba` functions in the corresponding member funtions.

In [8]:
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier


class Classifier(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, X, y):
        self.clf = RandomForestClassifier(
            n_estimators=2, max_leaf_nodes=2, random_state=61)
        self.clf.fit(X, y)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)


Let's try it on the training data, transformed into features. The output is a probability array, each row is 19 dimensional since we have 19 classes.

In [9]:
clf = Classifier()
clf.fit(X_array, y_array)
y_proba_array = clf.predict_proba(X_array)
y_proba_array.shape

(4560, 19)

## Scoring

To score, we forst convert the probability array into a label array by taking the label that received the highest probability.

In [10]:
y_pred_array = [problem._prediction_label_names[y] for y in np.argmax(y_proba_array, axis=1)]

Then we can compute the accuracy using `accuracy_score` of scikit learn.

In [11]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_array, y_pred_array)

0.10482456140350878

The following cell will evaluate the workflow on the test data.

In [12]:
X_test_df, y_test_array = problem.get_test_data()
X_test_array = fe.transform(X_test_df)
y_test_proba_array = clf.predict_proba(X_test_array)
y_test_pred_array = [problem._prediction_label_names[y] for y in np.argmax(y_test_proba_array, axis=1)]
accuracy_score(y_test_array, y_test_pred_array)

0.10478468899521531

## Local testing (before submission)

You can start playing with the cells above, modify the feature extractor and the classifier and evaluate the accuracy of the workflow. However, it is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. First you should save your feature extractor and classifier classes into [`submissions/starting_kit/feature_extractor.py`](/edit/submissions/starting_kit/feature_extractor.py) and [`submissions/starting_kit/classifier.py`](/edit/submissions/starting_kit/classifier.py), respectively. Then run the `ramp_test_submission` script either at the command line or here. This script executes a more robust cross validation on the training set, defined in `problem.py`. If this test runs without error, you can submit your feature extractor and classifier to [ramp.studio](http://www.ramp.studio/problems/radar_trajectories).

In [13]:
!ramp_test_submission

Testing Aircraft classification from radar trajectories
Reading train and test files from ./data ...
Reading cv ...
Training ./submissions/starting_kit ...
CV fold 0
	score    nll    acc
	train  2.489  0.107
	valid  2.554  0.098
	test   2.600  0.104
CV fold 1
	score    nll    acc
	train  2.490  0.104
	valid  2.496  0.109
	test   2.547  0.104
CV fold 2
	score    nll    acc
	train  2.506  0.104
	valid  2.477  0.109
	test   2.538  0.104
CV fold 3
	score    nll    acc
	train  2.505  0.104
	valid  2.542  0.109
	test   2.694  0.104
CV fold 4
	score    nll    acc
	train  2.490  0.104
	valid  2.519  0.109
	test   2.539  0.104
CV fold 5
	score    nll    acc
	train  2.508  0.104
	valid  2.537  0.109
	test   2.670  0.104
CV fold 6
	score    nll    acc
	train  2.498  0.107
	valid  2.486  0.098
	test   2.545  0.104
CV fold 7
	score    nll    acc
	train  2.493  0.107
	valid  2.483  0.114
	test   2.541  0.111
----------------------------
Mean CV scores
----------------------------
	score             

You can also edit and test other submissions saved into `submissions/<submission_name>`. For example, there is a submission in `submissions/more_features` using a bigger random forest and more features. You can test it using the following command.

In [14]:
!ramp_test_submission --submission more_features

Testing Aircraft classification from radar trajectories
Reading train and test files from ./data ...
Reading cv ...
Training ./submissions/more_features ...
CV fold 0
	score    nll    acc
	train  1.042  0.780
	valid  1.739  0.429
	test   1.707  0.450
CV fold 1
	score    nll    acc
	train  1.067  0.757
	valid  1.529  0.486
	test   1.737  0.462
CV fold 2
	score    nll    acc
	train  1.065  0.766
	valid  1.677  0.427
	test   1.693  0.447
CV fold 3
	score    nll    acc
	train  1.070  0.752
	valid  1.565  0.495
	test   1.660  0.457
CV fold 4
	score    nll    acc
	train  1.061  0.748
	valid  1.559  0.472
	test   1.646  0.462
CV fold 5
	score    nll    acc
	train  1.051  0.776
	valid  1.592  0.453
	test   1.724  0.447
CV fold 6
	score    nll    acc
	train  1.059  0.776
	valid  1.640  0.385
	test   1.711  0.444
CV fold 7
	score    nll    acc
	train  1.046  0.774
	valid  1.597  0.440
	test   1.630  0.476
----------------------------
Mean CV scores
----------------------------
	score            

## Submitting to [ramp.studio](http://ramp.studio)

Once you found a good feature extractor and classifier, you can submit them to [ramp.studio](http://www.ramp.studio). First, if it is your first time using RAMP, [sign up](http://www.ramp.studio/sign_up), otherwise [log in](http://www.ramp.studio/login). Then find an open event on the particular problem, for example, the event [radar_trajectories](https://www.ramp.studio/events/radar_trajectories_mines_201718) for this RAMP. Sign up for the event. Both signups are controled by RAMP administrators, so there **can be a delay between asking for signup and being able to submit**.

Once your signup request is accepted, you can go to your [sandbox](http://www.ramp.studio/events/radar_trajectories_mines_201718/sandbox) and copy-paste (or upload) [`feature_extractor.py`](/edit/submissions/starting_kit/feature_extractor.py) and [`classifier.py`](/edit/submissions/starting_kit/classifier.py) from `submissions/starting_kit`. Save it, rename it, then submit it. The submission is trained and tested on our backend in the same way as `ramp_test_submission` does it locally. While your submission is waiting in the queue and being trained, you can find it in the "New submissions (pending training)" table in [my submissions](http://www.ramp.studio/events/radar_trajectories_mines_201718/my_submissions). Once it is trained, you get a mail, and your submission shows up on the [public leaderboard](http://www.ramp.studio/events/radar_trajectories_mines_201718/leaderboard). 
If there is an error (despite having tested your submission locally with `ramp_test_submission`), it will show up in the "Failed submissions" table in [my submissions](http://www.ramp.studio/events/radar_trajectories_mines_201718/my_submissions). You can click on the error to see part of the trace.

After submission, do not forget to give credits to the previous submissions you reused or integrated into your submission.

The data set we use at the backend is usually different from what you find in the starting kit, so the score may be different.

The usual way to work with RAMP is to explore solutions, add feature transformations, select models, perhaps do some AutoML/hyperopt, etc., _locally_, and checking them with `ramp_test_submission`.

## More information

You can find more information in the [wiki](https://github.com/paris-saclay-cds/ramp-workflow/wiki) of the `ramp-workflow library`.

## Contact

Don't hesitate to [contact us](mailto:admin@ramp.studio?subject=radar trajectory notebook).